## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [6]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters = 16, kernel_size = (3, 3), padding = 'same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))


#卷積組合
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), padding = 'same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters = 64, kernel_size = (3, 3), padding = 'same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units = 64)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 7s 133us/step - loss: 1.3899 - acc: 0.5152
Epoch 2/100
50000/50000 [==============================] - 6s 114us/step - loss: 1.0011 - acc: 0.6483
Epoch 3/100
50000/50000 [==============================] - 6s 120us/step - loss: 0.8817 - acc: 0.6896
Epoch 4/100
50000/50000 [==============================] - 6s 121us/step - loss: 0.7973 - acc: 0.7204
Epoch 5/100
50000/50000 [==============================] - 6s 116us/step - loss: 0.7446 - acc: 0.7397
Epoch 6/100
50000/50000 [==============================] - 6s 120us/step - loss: 0.7019 - acc: 0.7577
Epoch 7/100
50000/50000 [==============================] - 6s 120us/step - loss: 0.6603 - acc: 0.7698
Epoch 8/100
50000/50000 [==============================] - 6s 120us/step - loss: 0.6264 - acc: 0.7800
Epoch 9/100
50000/50000 [==============================] - 6s 123us/step - loss: 0.5891 - acc: 0.7926
Epoch 10/100
50000/50000 [==============================] - 6s 118us/step - loss: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [9]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[4.6242676e-03, 6.9689139e-09, 4.7699979e-04, 8.1158632e-01,
        1.6470790e-01, 5.6092663e-06, 1.8015893e-02, 5.7017739e-04,
        1.2872265e-05, 3.7566027e-12]], dtype=float32)